<a href="https://colab.research.google.com/github/Choiyh1116/AI_capstone/blob/%EB%B3%B4%EC%9D%B4%EC%A7%80%EC%95%8A%EB%8A%94%EA%B2%80%EC%9D%B4%EA%B0%80%EC%9E%A5%EB%AC%B4%EC%84%9C%EC%9A%B4%EB%B2%95/code/videoToCsv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#%%time #코드 러닝타임 확인용

"""
현재 1초(30프레임)마다 넘겨보는 중 > frameGap 조절
비디오 경로 확인
colab에서도 csv파일 생성 잘 되는것 확인
"""

import torch
import cv2
import os
import numpy as np
import matplotlib.pylab as plt

#비디오 경로설정 필수
vidcap = cv2.VideoCapture('drive/MyDrive/Videos/dogs.mp4')
methods = ['CORREL', 'CHISQR', 'INTERSECT', 'BHATTACHARYYA', 'EMD']
hists = []

#csv파일관련, csv파일 이름, 경로설정 확인
cor, chi, ist, bha, emd, sct = list(), list(), list(), list(), list(), list()
filecsv = open("testcsv1.csv", "w")

#영상 못불러오면 코드종료
if not vidcap.isOpened():
    print("Could not Open")
    exit(0)

count = 0

#각각 영상 초당프레임, 총 프레임 수, 계산된 영상 길이(초단위)
fps = int(vidcap.get(cv2.CAP_PROP_FPS))
totalf = vidcap.get(cv2.CAP_PROP_FRAME_COUNT)
totalt = totalf/fps
print("FPS : %d\t영상길이 : %f"%(fps,totalt))

#코드돌리는시간이 너무 길다면, 조절한다 -> frameGap
frameGap = int(fps/1)
gapCount = int(frameGap-1)
#코드돌리는시간이 너무 길다면, 조절한다 -> maxFrame
maxFrame = int(totalf/1)

#그래프관련
xchuk = list()
howmany = 5  #x축 그래프 분할 확인용, 현재 5분할
graphtime = maxFrame/fps
itimes = int(graphtime/howmany)

#이미지 크기 조절 관련, 러닝타임 단축 기대, %%time으로 확인한 결과 효과없음, 오히려 지연됨
"""width = vidcap.get(cv2.CAP_PROP_FRAME_WIDTH)
height = vidcap.get(cv2.CAP_PROP_FRAME_HEIGHT)
scale = 4"""

while(vidcap.isOpened()): 
    if(gapCount==0):
        ret, image = vidcap.read()
        gapCount = int(frameGap-1)
    else:
        ret = vidcap.grab()
        gapCount -= 1
        continue
    if ret:
        #image = cv2.resize(image, (int(width/scale), int(height/scale)))
        
        if(int (vidcap.get(1) % frameGap == 0)):
            if(int (vidcap.get(1) > maxFrame)):
                break
            hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
            hist = cv2.calcHist([hsv], [0, 1], None, [180, 256], [0, 180, 0, 256])
            cv2.normalize(hist, hist, 0, 1, cv2.NORM_MINMAX)
            hists.append(hist)
                
            if(len(hists)==2):
                query = hists[0]
                for index, name in enumerate(methods):
    
                    for i, histogram in enumerate(hists):
                        if(i == 0):
                            continue
                        compare = cv2.compareHist(query, histogram, index) 
        
                        if(index == cv2.HISTCMP_INTERSECT):
                            compare = compare/np.sum(query)
                        
                        if(name == 'CORREL'):
                            cor.append(compare)
                        if(name == 'CHISQR'):
                            chi.append(compare)
                            xchuk.append(int(vidcap.get(1))/30)
                        if(name == 'INTERSECT'):
                            ist.append(compare)
                        if(name == 'BHATTACHARYYA'):
                            bha.append(compare)
                        if(name == 'EMD'):
                            emd.append(compare)
                            sct.append(0)
                            
                del(hists[0])
            
            
            count += 1
    else:
        break
        
filecsv.write('CORREL,CHISQR,INTERSECT,BHATTACHARYYA,EMD,Screen_Transition\n')
for i in range(len(cor)):
    filecsv.write(str(cor[i]) + ',' + str(chi[i]) + ',' + str(ist[i]) + ',' + str(bha[i]) + ',' + str(emd[i]) + ',' + str(sct[i]) + '\n')
    
filecsv.close()

"""for i in range(0,howmany):
    plt.xlim(i*itimes,(i+1)*itimes)
    plt.plot(xchuk, cor)
    plt.show()
    plt.xlim(i*itimes,(i+1)*itimes)
    plt.plot(xchuk, chi)
    plt.show()
    plt.xlim(i*itimes,(i+1)*itimes)
    plt.plot(xchuk, ist)
    plt.show()
    plt.xlim(i*itimes,(i+1)*itimes)
    plt.plot(xchuk, bha)
    plt.show()
    plt.xlim(i*itimes,(i+1)*itimes)
    plt.plot(xchuk, emd)
    plt.show()

plt.plot(xchuk, cor)
plt.show()
plt.plot(xchuk, chi)
plt.show()
plt.plot(xchuk, ist)
plt.show()
plt.plot(xchuk, bha)
plt.show()
plt.plot(xchuk, emd)
plt.show()"""

del(cor,chi,ist,bha,emd,sct)

vidcap.release()

print("코드 구동 종료")

FPS : 30	영상길이 : 202.266667
코드 구동 종료
